In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
from tqdm import tqdm

driver=webdriver.Chrome()
driver.get("https://www.amazon.in/s?k=mobiles&crid=3GGHIO8WGFWGJ&sprefix=%2Caps%2C230&ref=nb_sb_ss_recent_1_0_recent")
html_data_1=BeautifulSoup(driver.page_source,'html.parser')
No_of_pages = html_data_1.find('span',{'class':'s-pagination-item s-pagination-disabled'}).text
titles=[]
images=[]
ratings=[]
prices=[]
product_urls=[]
for i in tqdm(range(int(No_of_pages))):
   url = 'https://www.amazon.in/s?k=mobiles&crid=2944PM084A0K4&sprefix=mobiles%2Caps%2C271&ref=nb_sb_noss_1&page='+str(i+1)
   driver.get(url)
   time.sleep(3)
   html_data_2=BeautifulSoup(driver.page_source,'html.parser')
   products=html_data_2.find_all('div',{'data-component-type':'s-search-result'})
   for product in products:
      title=product.find('h2',{'class':'a-size-mini a-spacing-none a-color-base s-line-clamp-2'}).text
      titles.append(title)
      img = product.find('img')['src']
      images.append(img)
      rating=product.find('span',{'class':'a-icon-alt'})
      if rating == None:
         rating='No Rating'
      else:
         rating=product.find('span',{'class':'a-icon-alt'}).text
      ratings.append(rating)
      price = product.find('span',{'class':'a-price-whole'})
      if price==None or product.find('span',{'aria-label':'Currently unavailable.'}) or product.find('span',{'class':'a-size-base a-color-secondary'}):
         price='No Price'
      else:
         price = '₹'+product.find('span',{'class':'a-price-whole'}).text
      prices.append(price)
      product_url=product.find('a')
      product_fullurl="https://amazon.in/"+product_url['href']
      product_urls.append(product_fullurl)

100%|██████████| 20/20 [01:57<00:00,  5.85s/it]


In [ ]:
data = pd.DataFrame({"titles":titles,"images":images,"ratings":ratings,"prices":prices,"product_urls":product_urls})
data.to_csv(r"C:\Users\sujay\Desktop\Chatbot\CSVFilesObtainedAfterWebscrapping\Amazon\amazon_reviews_final.csv")

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
driver=webdriver.Chrome()
review_links=[]
for i in range(len(product_urls)):
    driver.get(product_urls[i])
    review_data = BeautifulSoup(driver.page_source, 'html.parser')
    reviewlink = review_data.find('a', {'data-hook': 'see-all-reviews-link-foot'})
    if reviewlink is None:
        link = 'No Link'
    else:
        link = "https://www.amazon.in" + reviewlink['href']
    review_links.append(link)

In [ ]:
data1=pd.DataFrame({'review':review_links})
data1.to_csv(r'C:\Users\sujay\Desktop\Chatbot\CSVFilesObtainedAfterWebscrapping\Amazon\reviewlinks.csv')

In [ ]:
df=pd.read_csv('reviewlinks.csv')
df=df[df.review!='No Link']
df.to_csv(r"C:\Users\sujay\Desktop\Chatbot\CSVFilesObtainedAfterWebscrapping\Amazon\NewreviewLinks.csv")

In [ ]:
new_df=pd.read_csv(r"C:\Users\sujay\Desktop\Chatbot\CSVFilesObtainedAfterWebscrapping\Amazon\NewreviewLinks.csv")


In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
# Path to your existing browser profile
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("user-data-dir=/path/to/your/profile")

# Set up the Chrome WebDriver with the profile
service = Service(ChromeDriverManager().install())

def scrape_amazon_reviews(product_url):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(product_url)
    
    reviews_text = []
    names = []
    rating_dates = []
    titles = []
    ratingsss=[]
    reviewproducttitles=[]
    while product_url is not None:
        time.sleep(5)  
        html_data = BeautifulSoup(driver.page_source, 'html.parser')
        reviews = html_data
        
        for review in reviews:
            all_reviews=review.find_all('div',{'data-hook':'review','class':'a-section review aok-relative'})

            for r in all_reviews:
                review_text = r.find('span', {'data-hook': 'review-body','class':'a-size-base review-text review-text-content'}).text.strip()
                reviews_text.append(review_text)
                name = r.find('span', {'class': 'a-profile-name'}).text.strip()
                names.append(name)
                ratingss = r.find('span', {'class': 'a-icon-alt'}).text
                ratingsss.append(ratingss)
                rating_date = r.find('span', {'data-hook': 'review-date'}).text
                rating_dates.append(rating_date)
                title = review.find('a', {'data-hook': 'review-title'})
                if title==None:
                   title='No Title'
                else:
                    title = review.find('a', {'data-hook': 'review-title'}).text.strip()
                titles.append(title[19:])

            reviewproducttitle = review.find('a', {'data-hook': 'product-link'})
            if reviewproducttitle==None:
                reviewproducttitle='No Product Title'
            else:
                reviewproducttitle = review.find('a', {'data-hook': 'product-link'})
            for _ in range(len(all_reviews)):
                reviewproducttitles.append(reviewproducttitle.text)
        url_check = html_data.find('li', {'class': 'a-last'})
        if url_check is None or url_check.a is None:
            product_url = None  # No more pages to navigate
        else:
            product_url = url_check.a['href']
            driver.find_element(By.LINK_TEXT,"Next page").click()
    
    driver.quit()
    return reviews_text, names, ratingsss, rating_dates, titles,reviewproducttitles


all_reviews_text_0 = []
all_names_0 = []
all_ratings_0 = []
all_rating_dates_0 = []
all_titles_0 = []
all_review_product_titles=[]
for url in new_df.review[0:101]:
    reviews_text, names, ratingsss, rating_dates, titles,reviewproducttitles = scrape_amazon_reviews(url)
    all_reviews_text_0.extend(reviews_text)
    all_names_0.extend(names)
    all_ratings_0.extend(ratingsss)
    all_rating_dates_0.extend(rating_dates)
    all_titles_0.extend(titles)
    all_review_product_titles.extend(reviewproducttitles)
    #Add a separator for each product
    all_reviews_text_0.append('-------')
    all_names_0.append('-------')
    all_ratings_0.append('-------')
    all_rating_dates_0.append('-------')
    all_titles_0.append('-------')
    all_review_product_titles.append('-------')


In [ ]:
# Creating a DataFrame
df = pd.DataFrame({
    'Product Title':all_review_product_titles,
    'Title': all_titles_0,
    'Name': all_names_0,
    'Ratings':all_ratings_0,
    'Rating Date': all_rating_dates_0,
    'Review Text': all_reviews_text_0
})


In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
# Path to your existing browser profile
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("user-data-dir=/path/to/your/profile")

# Set up the Chrome WebDriver with the profile
service = Service(ChromeDriverManager().install())

def scrape_amazon_reviews(product_url):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(product_url)
    
    reviews_text = []
    names = []
    rating_dates = []
    titles = []
    ratingsss=[]
    reviewproducttitles=[]
    while product_url is not None:
        time.sleep(5)  
        html_data = BeautifulSoup(driver.page_source, 'html.parser')
        reviews = html_data
        
        for review in reviews:
            all_reviews=review.find_all('div',{'data-hook':'review','class':'a-section review aok-relative'})

            for r in all_reviews:
                review_text = r.find('span', {'data-hook': 'review-body','class':'a-size-base review-text review-text-content'}).text.strip()
                reviews_text.append(review_text)
                name = r.find('span', {'class': 'a-profile-name'}).text.strip()
                names.append(name)
                ratingss = r.find('span', {'class': 'a-icon-alt'}).text
                ratingsss.append(ratingss)
                rating_date = r.find('span', {'data-hook': 'review-date'}).text
                rating_dates.append(rating_date)
                title = review.find('a', {'data-hook': 'review-title'})
                if title==None:
                   title='No Title'
                else:
                    title = review.find('a', {'data-hook': 'review-title'}).text.strip()
                titles.append(title[19:])

            reviewproducttitle = review.find('a', {'data-hook': 'product-link'})
            if reviewproducttitle==None:
                reviewproducttitle='No Product Title'
            else:
                reviewproducttitle = review.find('a', {'data-hook': 'product-link'})
            for _ in range(len(all_reviews)):
                reviewproducttitles.append(reviewproducttitle.text)
        url_check = html_data.find('li', {'class': 'a-last'})
        if url_check is None or url_check.a is None:
            product_url = None  # No more pages to navigate
        else:
            product_url = url_check.a['href']
            driver.find_element(By.LINK_TEXT,"Next page").click()
    
    driver.quit()
    return reviews_text, names, ratingsss, rating_dates, titles,reviewproducttitles


all_reviews_text_1 = []
all_names_1 = []
all_ratings_1 = []
all_rating_dates_1 = []
all_titles_1 = []
all_review_product_titles=[]
for url in new_df.review[101:201]:
    reviews_text, names, ratingsss, rating_dates, titles,reviewproducttitles = scrape_amazon_reviews(url)
    all_reviews_text_1.extend(reviews_text)
    all_names_1.extend(names)
    all_ratings_1.extend(ratingsss)
    all_rating_dates_1.extend(rating_dates)
    all_titles_1.extend(titles)
    all_review_product_titles.extend(reviewproducttitles)
    #Add a separator for each product
    all_reviews_text_1.append('-------')
    all_names_1.append('-------')
    all_ratings_1.append('-------')
    all_rating_dates_1.append('-------')
    all_titles_1.append('-------')
    all_review_product_titles.append('-------')


In [ ]:
df2 = pd.DataFrame({
    'Product Title':all_review_product_titles,
    'Title': all_titles_1,
    'Name': all_names_1,
    'Ratings':all_ratings_1,
    'Rating Date': all_rating_dates_1,
    'Review Text': all_reviews_text_1
})

In [12]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
# Path to your existing browser profile
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("user-data-dir=/path/to/your/profile")

# Set up the Chrome WebDriver with the profile
service = Service(ChromeDriverManager().install())

def scrape_amazon_reviews(product_url):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(product_url)
    
    reviews_text = []
    names = []
    rating_dates = []
    titles = []
    ratingsss=[]
    reviewproducttitles=[]
    while product_url is not None:
        time.sleep(5)  
        html_data = BeautifulSoup(driver.page_source, 'html.parser')
        reviews = html_data
        
        for review in reviews:
            all_reviews=review.find_all('div',{'data-hook':'review','class':'a-section review aok-relative'})

            for r in all_reviews:
                review_text = r.find('span', {'data-hook': 'review-body','class':'a-size-base review-text review-text-content'}).text.strip()
                reviews_text.append(review_text)
                name = r.find('span', {'class': 'a-profile-name'}).text.strip()
                names.append(name)
                ratingss = r.find('span', {'class': 'a-icon-alt'}).text
                ratingsss.append(ratingss)
                rating_date = r.find('span', {'data-hook': 'review-date'}).text
                rating_dates.append(rating_date)
                title = review.find('a', {'data-hook': 'review-title'})
                if title==None:
                   title='No Title'
                else:
                    title = review.find('a', {'data-hook': 'review-title'}).text.strip()
                titles.append(title[19:])

            reviewproducttitle = review.find('a', {'data-hook': 'product-link'})
            if reviewproducttitle==None:
                reviewproducttitle='No Product Title'
            else:
                reviewproducttitle = review.find('a', {'data-hook': 'product-link'})
            for _ in range(len(all_reviews)):
                reviewproducttitles.append(reviewproducttitle.text)
        url_check = html_data.find('li', {'class': 'a-last'})
        if url_check is None or url_check.a is None:
            product_url = None  # No more pages to navigate
        else:
            product_url = url_check.a['href']
            driver.find_element(By.LINK_TEXT,"Next page").click()
    
    driver.quit()
    return reviews_text, names, ratingsss, rating_dates, titles,reviewproducttitles


all_reviews_text_2 = []
all_names_2 = []
all_ratings_2 = []
all_rating_dates_2 = []
all_titles_2 = []
all_review_product_titles=[]
for url in new_df.review[201:]:
    reviews_text, names, ratingsss, rating_dates, titles ,reviewproducttitles= scrape_amazon_reviews(url)
    all_reviews_text_2.extend(reviews_text)
    all_names_2.extend(names)
    all_ratings_2.extend(ratingsss)
    all_rating_dates_2.extend(rating_dates)
    all_titles_2.extend(titles)
    all_review_product_titles.extend(reviewproducttitles)
    #Add a separator for each product
    all_reviews_text_2.append('-------')
    all_names_2.append('-------')
    all_ratings_2.append('-------')
    all_rating_dates_2.append('-------')
    all_titles_2.append('-------')
    all_review_product_titles.append('-------')


In [ ]:
# Creating a DataFrame
df3 = pd.DataFrame({
    'Product Title':all_review_product_titles,
    'Title': all_titles_2,
    'Name': all_names_2,
    'Ratings':all_ratings_2,
    'Rating Date': all_rating_dates_2,
    'Review Text': all_reviews_text_2
})

In [ ]:
merged_df = pd.concat([df, df2, df3], ignore_index=True)
merged_df

,Unnamed: 0,Product Title,Title,Name,Ratings,Rating Date,Review Text
0,0,Redmi 13 5g Black Diamond 6GB 128GB,Great phone great specs,AD Pal,5.0 out of 5 stars,Reviewed in India on 29 July 2024,"The Worth it phone, I've tried many other bran..."
1,1,Redmi 13 5g Black Diamond 6GB 128GB,Great phone great specs,Raja,4.0 out of 5 stars,Reviewed in India on 17 August 2024,"I used this mobile, past 30 days. It worth for..."
2,2,Redmi 13 5g Black Diamond 6GB 128GB,Great phone great specs,Bablu Kumar Mandal,4.0 out of 5 stars,Reviewed in India on 8 August 2024,This is really a good 5G phone in this budget ...
3,3,Redmi 13 5g Black Diamond 6GB 128GB,Great phone great specs,Rahul Kumar Sharma,5.0 out of 5 stars,Reviewed in India on 10 August 2024,"The phone has an eye-catching, contemporary de..."
4,4,Redmi 13 5g Black Diamond 6GB 128GB,Great phone great specs,Dinesh Bhika Bhadane,5.0 out of 5 stars,Reviewed in India on 15 August 2024,Thanks Amazon for the right products with good...
...,...,...,...,...,...,...,...
28170,12126,Spigen Ultra Hybrid Back Cover Case for iPhone...,Durable cover,Sada,3.0 out of 5 stars,Reviewed in India on 19 October 2021,"The item is good for about 6 months, the slowl..."
28171,12127,Spigen Ultra Hybrid Back Cover Case for iPhone...,Durable cover,Pratik Pandey,4.0 out of 5 stars,Reviewed in India on 12 July 2021,Very Nice product. Perfectly fir to iphone Xs.
28172,12128,Spigen Ultra Hybrid Back Cover Case for iPhone...,Durable cover,Srikanth,4.0 out of 5 stars,Reviewed in India on 17 September 2020,Quality good but within a week sides turned in...
28173,12129,Spigen Ultra Hybrid Back Cover Case for iPhone...,Durable cover,Keshav sharma,4.0 out of 5 stars,Reviewed in India on 18 June 2021,Turns yellow after 6 months of use


In [ ]:
merged_df.to_csv(r"C:\Users\sujay\Desktop\Chatbot\CSVFilesObtainedAfterWebscrapping\Amazon\amazon_reviews_final.csv")

353